# Quick intro to Dash

[plotly dash](https://dash.plotly.com/) is a framework to build app in python.

As it is developped by the same groups, it works hand-in-hand with plotly.

Using an app framework such as dash will help you create a data app with:
 * more interactivity components than simple plotly
 * do computation with python during usage, rather than ahead of time
 * finely control the appearance of your app
 
However, whereas before we could just export the interactive vizualisation as a **static html** that can be easily sent to anyone or hosted freely on something like a github page,

now if you want to share your app on the internet it requires that you set-up an actual **server**.

Options exists, many of them with an attached price tag. 
Beware it is a moving landscape, with services regularly switching from free to paying over the years.

Here are a few:
 * [dash enterprise](https://plotly.com/dash/) : plotly dash service, which includes deployment and also additional modules.
 * https://www.heroku.com : no longer any free option
 * https://render.com : free option for hosting app with 512MB of RAM & 0.1 CPU

Last but not least, do not hesitate to **contact your institute/university/enterprise IT team**, or HPC team: it is very likely they have some idea on how you can securely host your app, and they may have some capacity to help you host it on your organization's resources (potentially for a fraction of the cost).


In any case, we advise you to :
 * think twice about the need for a server architecture (would one of the advanced plotly trick be enough?)
 * inquire about hosting solution **before** you do your app
 
 

## A first dash app

a dash app can be built in the notebook or in an independent python file.

 * the notebook is great for quick debugging and design
 * the independent python file is the more "classical" way and what you'll need for actual deployment in the end

In [ ]:
import pandas as pd 

In [ ]:
## reading data from the 1880 Swiss census
df = pd.read_csv('../course1/data/census1880_fractions.csv', index_col=0)
df.head()

In [ ]:
df['canton name'].unique()

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import numpy as np

## reading data from the 1880 Swiss census
df = pd.read_csv('../course1/data/census1880_fractions.csv')

app = Dash()

app.layout = [
    html.H1(children='1880 swiss census', style={'textAlign':'center'}),
    dcc.Dropdown(df['canton name'].unique(), 'Basel-Stadt', id='dropdown-selection'),
    dcc.Graph(id='graph-content')
]

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    
    dff = df.loc[ df['canton name']==value , : ]
    
    fig = px.histogram( np.log10( dff.Total ) ) # using numpy log10 here because plotly histogram and axis log scale does not work
    fig.update_xaxes(title_text = 'log10 number of inhabitants')
    
    return fig

if __name__ == '__main__':
    app.run(debug=True , host='127.0.0.1')


More elements can be linked together:


In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import numpy as np

## reading data from the 1880 Swiss census
df = pd.read_csv('../course1/data/census1880_fractions.csv')

app = Dash()

app.layout = html.Div([
    html.H1(children='1880 swiss census', style={'textAlign':'center'}),
    
    html.Div([
        html.Div(children='''Pick a canton:'''),
        dcc.Dropdown(df['canton name'].unique(), 'Basel-Stadt', id='canton-selection')
    ],style={'width': '48%', 'display': 'inline-block'}),
    
    html.Div([
        html.Div(children='''Pick a color:'''),
        dcc.Dropdown(['blue','green','grey'], 'blue', id='color-selection')
    ],style={'width': '48%', 'display': 'inline-block'}),
    
    dcc.Graph(id='graph-content')
])

@callback(
    Output('graph-content', 'figure'),
    Input('canton-selection', 'value'), ## the names are all value, the order is what matters
    Input('color-selection', 'value')
)
def update_graph(canton, color):
    
    dff = df.loc[ df['canton name']==canton , : ]
    
    fig = px.histogram( np.log10( dff.Total ) , 
                       color_discrete_sequence=[color] )
    fig.update_xaxes(title_text = 'log10 number of inhabitants')
    
    return fig

if __name__ == '__main__':
    app.run(debug=True , host='127.0.0.1')



**Exercise:** 
 modify the app above so there is a radio-items letting the user choose between a linear and a log scale.
 
Help yourself with the [radio-items help page](https://dash.plotly.com/dash-core-components/radioitems).
 

In [ ]:
# %load solutions/solution_dash_app_radioitem.py


Many components have options which can modify their behavior, 

for example, the `multi` option of `dcc.Dropdown`:

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import numpy as np

## reading data from the 1880 Swiss census
df = pd.read_csv('../course1/data/census1880_fractions.csv')

app = Dash()

app.layout = [
    html.H1(children='1880 swiss census', style={'textAlign':'center'}),
    dcc.Dropdown(df['canton name'].unique(), 'Basel-Stadt', id='dropdown-selection' , 
                 multi=True), ## allowing multiple selections
    dcc.Graph(id='graph-content')
]

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    
    # when there is a single canton selected, value is a string
    # when there is several cantons selected, value is a list
    # we have to account for that
    if isinstance(value, str): 
        value = [value]
    
    dff = df.loc[ df['canton name'].isin( value ) , : ]
    
    fig = px.histogram( np.log10( dff.Total ) , 
                       color = dff['canton name'] ) # using numpy log10 here because plotly histogram and axis log scale does not work
    fig.update_xaxes(title_text = 'log10 number of inhabitants')
    
    return fig

if __name__ == '__main__':
    app.run(debug=True , host='127.0.0.1')



Useful link: [dash core component](https://dash.plotly.com/dash-core-components)

## App layout

There's several way you can build your app layout,

* using html components (as we-ve done until now)
* using dash bootstrap layout components
* *dash design kit (dash enterprise version only)*


[bootstrap](https://dash-bootstrap-components.opensource.faculty.ai/) offers many components to help you build the dash app you want, we will focus here mainly on their [layout elements](https://dash-bootstrap-components.opensource.faculty.ai/docs/components/layout/) which let you easily build a something from rows and columns.



**important** for the bootstrap component to work you must apply a compatible theme. You can browse themes at : [https://bootswatch.com/](https://bootswatch.com/)

In [ ]:
import dash_bootstrap_components as dbc
from dash import html

external_stylesheets = [dbc.themes.BOOTSTRAP] # try other theme, such as dbc.themes.QUARTZ
app = Dash( external_stylesheets=external_stylesheets )

# App layout
app.layout = dbc.Container(
    [
        dbc.Row(dbc.Col(html.Div("A single column"))),
        dbc.Row(
            [
                dbc.Col(html.Div("One of three columns")),
                dbc.Col(html.Div("One of three columns")),
                dbc.Col(html.Div("One of three columns")),
            ]
        ),
    ]
)
app.run(debug=True , host='127.0.0.1')

It is easier to see what happens here with `dbc.Alert`:

In [ ]:
import dash_bootstrap_components as dbc
from dash import html

external_stylesheets = [dbc.themes.BOOTSTRAP] # try other theme, such as dbc.themes.QUARTZ
app = Dash( external_stylesheets=external_stylesheets )

# App layout
app.layout = dbc.Container(
    [
        dbc.Row(dbc.Col(dbc.Alert("A single column"))),
        dbc.Row(
            [
                dbc.Col(dbc.Alert("One of three columns")),
                dbc.Col(dbc.Alert("One of three columns")),
                dbc.Col(dbc.Alert("One of three columns")),
            ]
        ),
    ]
)
app.run(debug=True , host='127.0.0.1')

From there, you can:

 * nest elements 
 * change the width (each row has a width of 12 which you can spread between columns)
 * change the alignment behavior in rows or columns
 * more options [online](https://dash-bootstrap-components.opensource.faculty.ai/docs/components/layout/)
    

In [ ]:
import dash_bootstrap_components as dbc
from dash import html

external_stylesheets = [dbc.themes.BOOTSTRAP] # try other theme, such as dbc.themes.QUARTZ
app = Dash( external_stylesheets=external_stylesheets )

# App layout
app.layout = dbc.Container(
    [
        dbc.Row(dbc.Col(dbc.Alert("A single column"))),
        dbc.Row(
            [
                dbc.Col(dbc.Alert("One of three columns") , width = 8),
                dbc.Col(dbc.Alert("") , width = 1),
                dbc.Col(
                    dbc.Container([  
                        dbc.Row(
                            dbc.Col(dbc.Alert("row of third column"))
                        ),
                        dbc.Row(
                            dbc.Col(dbc.Alert("row of third column"))
                        )
                    ]), width = 3 )
            ],
            align="center",
        ),
    ]
)
app.run(debug=True , host='127.0.0.1')

**Exercise**:

modify the app code below such that you have the 
graph in one big column and the dropdown and radioItem on a smaller column beside.

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import numpy as np

## reading data from the 1880 Swiss census
df = pd.read_csv('../course1/data/census1880_fractions.csv')

app = Dash()

app.layout = html.Div([
    html.H1(children='1880 swiss census', style={'textAlign':'center'}),
    
    html.Div([
        html.Div(children='''Pick a canton:'''),
        dcc.Dropdown(df['canton name'].unique(), 'Basel-Stadt', id='canton-selection'),
        dcc.RadioItems(['Log','Linear'], 'Log', inline=True, id='scale-selection')
    ]),
    
    dcc.Graph(id='graph-content')
])

@callback(
    Output('graph-content', 'figure'),
    Input('canton-selection', 'value'), ## the names are all value, the order is what matters
    Input('scale-selection', 'value')
)
def update_graph(canton, scale):
    
    
    dff = df.loc[ df['canton name']==canton , : ]
    
    data = dff.Total
    xlabel = 'number of inhabitants'
    if scale == "Log":
        data = np.log10( data )
        xlabel = 'log10 number of inhabitants'
    
    fig = px.histogram( data )
    fig.update_xaxes(title_text = xlabel)
    
    return fig

if __name__ == '__main__':
    app.run(debug=True , host='127.0.0.1')



Solution:

In [ ]:
# %load solutions/solution_dash_app_bootstrap.py

## mouse behavior

The nice thing with Dash is its interaction with plotly graph to gather which points are hovered upon, clicked, selected, ...

In [ ]:
from dash import Dash, dcc, html, Input, Output, callback

import plotly.express as px

import json
import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(external_stylesheets=external_stylesheets)

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}


## toy dataset
df = pd.DataFrame({
    "x": [1,2,1,2],
    "y": [1,2,3,4],
    "customdata": ['a','b','c','d'],
    "fruit": ["apple", "apple", "orange", "orange"]
})

fig = px.scatter(df, x="x", y="y", color="fruit", custom_data=["customdata"])

fig.update_layout(clickmode='event+select')

fig.update_traces(marker_size=20)

app.layout = html.Div([
    dcc.Graph(
        id='basic-interactions',
        figure=fig
    ),

    html.Div(className='row', children=[
        html.Div([
            dcc.Markdown("""
                **Hover Data**

                Mouse over values in the graph.
            """),
            html.Pre(id='hover-data', style=styles['pre'])
        ], className='three columns'),

        html.Div([
            dcc.Markdown("""
                **Click Data**

                Click on points in the graph.
            """),
            html.Pre(id='click-data', style=styles['pre']),
        ], className='three columns'),

        html.Div([
            dcc.Markdown("""
                **Selection Data**

                Choose the lasso or rectangle tool in the graph's menu
                bar and then select points in the graph.

                Note that if `layout.clickmode = 'event+select'`, selection data also
                accumulates (or un-accumulates) selected data if you hold down the shift
                button while clicking.
            """),
            html.Pre(id='selected-data', style=styles['pre']),
        ], className='three columns'),

        html.Div([
            dcc.Markdown("""
                **Zoom and Relayout Data**

                Click and drag on the graph to zoom or click on the zoom
                buttons in the graph's menu bar.
                Clicking on legend items will also fire
                this event.
            """),
            html.Pre(id='relayout-data', style=styles['pre']),
        ], className='three columns')
    ])
])



## the different callbacks have the same input component, but grab different aspects of it
@callback(
    Output('hover-data', 'children'),
    Input('basic-interactions', 'hoverData')) 
def display_hover_data(hoverData):
    return json.dumps(hoverData, indent=2)


@callback(
    Output('click-data', 'children'),
    Input('basic-interactions', 'clickData'))
def display_click_data(clickData):
    return json.dumps(clickData, indent=2)


@callback(
    Output('selected-data', 'children'),
    Input('basic-interactions', 'selectedData'))
def display_selected_data(selectedData):
    return json.dumps(selectedData, indent=2)


@callback(
    Output('relayout-data', 'children'),
    Input('basic-interactions', 'relayoutData'))
def display_relayout_data(relayoutData):
    return json.dumps(relayoutData, indent=2)


if __name__ == '__main__':
    app.run(debug=True, host='127.0.0.1')
